In [9]:
import paramiko as pm
import socket as socket
import numpy as np
import os

class Server:
    def __init__(self, username: str, host: str, home_path_remote: str):
        self.host = host
        self.user = username
        self.home_path = home_path_remote # SFTP home path

    def connect(self):
        '''Connects to server and returns SFTP and transport objects'''
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.connect((self.host, 22))
        ts = pm.Transport(sock)
        ts.start_client(timeout=10)
        ts.auth_interactive_dumb(self.user)
        print(ts)
        sftp = pm.SFTPClient.from_transport(ts)
        print(sftp)
        return sftp, ts
    
    def list_folders(self, sftp:object):
        '''Returns list of folders in SFTP home directory'''
        return sftp.listdir(self.home_path)
    
    def download_folder(self, sftp: object, job_name:str, temp_localfolder):
        '''Download all files in job folder to local temporary folder'''
        # Source and target folders specification
        os.mkdir(f'{temp_localfolder}{job_name}')
        source_folder=f'/lustre/home/st/st_us-031400/st_st179390/{job_name}/'
        inbound_files=sftp.listdir(source_folder)

        # Download all files from that path
        for file in inbound_files:
            filepath = f'{source_folder}{file}'
            localpath = f'{temp_localfolder}{job_name}/{file}'
            sftp.get(filepath, localpath)

In [10]:
temp_localfolder = '/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_temporary/'
#job_name = 'MD_Test01_420eV_3x5x6_bulk_4b'

In [11]:
# Initialize Server
server = Server('st_st179390','justus2.uni-ulm.de', '/lustre/home/st/st_us-031400/st_st179390/')
sftp, ts = server.connect()

Your OTP: Password: <paramiko.Transport at 0x1784ffd0 (cipher aes128-ctr, 128 bits) (active; 0 open channel(s))>


In [12]:
# List directories
jobs = server.list_folders(sftp)

In [13]:
jobs

['.bashrc',
 'bulk_LPS_mesh_LREAL_6x10x12_500eV',
 '.bash_profile',
 'bulk_LPS_mesh_LREAL_2x2x4_500eV',
 'bulk_LPS_mesh_6x8x13_500eV',
 'bulk_LPS_6x10x12_500eV',
 'bulk_LPS_mesh_6x6x6_500eV',
 'bulk_LPS_6x10x12_340eV',
 'bulk_LPS_mesh_LREAL_8x10x14_500eV',
 'bulk_LPS_mesh_4x4x4_500eV',
 'bulk_LPS_mesh_LREAL_7x9x15_500eV',
 'bulk_LPS_mesh_1x1x1_500eV',
 'bulk_LPS_mesh_LREAL_4x4x4_500eV',
 'test_LH5',
 'bulk_LPS_LREAL_auto_mesh_1x1x1_500eV',
 'bulk_LPS_12_x_10_x_6_280eV_Ex_4b',
 'MD_Test_TEBEG_1200k_3x5x6_460eV_bulk_4b',
 'bulk_LPS_mesh_5x7x11_500eV',
 'codes',
 'bulk_LPS_6x10x12_620eV',
 'bulk_LPS_mesh_LREAL_1x2x2_500eV',
 'bulk_LPS_6x10x12_740eV',
 'batch_scripts',
 'bulk_LPS_mesh_1x2x2_500eV',
 'bulk_LPS_6x10x12_700eV',
 'bulk_LPS_12_x_10_x_6_280eV_Ex_4c',
 'bulk_LPS_LREAL_auto_mesh_2x2x4_500eV',
 'bulk_LPS_6x10x12_100eV',
 'bulk_LPS_mesh_LREAL_8x8x8_500eV',
 'MD_Test_PMASS_500amu_3x5x6_460eV_bulk_4b',
 'bulk_LPS_mesh_8x8x8_500eV',
 'bulk_LPS_mesh_4x5x8_500eV',
 'bulk_LPS_mesh_7x9x15_

In [14]:
# filter jobs
jobs = list(filter(lambda x: x.startswith('MD_Test_TEBEG'),jobs))
n_jobs = len(jobs)

In [517]:
# filter jobs (convergence testing)
jobs_filtered = []

for job in jobs:
    if 'bulk_LPS_mesh' in job or 'bulk_LPS_LREAL' in job:
        jobs_filtered.append(job)

In [519]:
jobs = jobs_filtered
n_jobs = len(jobs)

In [15]:
# Download files
for job in jobs:
    server.download_folder(sftp, job, temp_localfolder)

In [16]:
# Terminate connection
if sftp: sftp.close()
if ts.active: ts.close()

In [23]:
# !Requires H5 output file!
#import py4vasp

#calc = py4vasp.Calculation.from_path(f'{temp_localfolder}{job_name}')
#dos = calc.dos.read()

In [508]:
import os
jobs = os.listdir(temp_localfolder)
#n_jobs = len(jobs)

In [520]:
import pandas as pd

entries = n_jobs*[0]

results = pd.DataFrame(
    {   
        'Job name': entries,
        'k-points': entries,
        '# k-points': entries,
        'Energy cutoff [eV]': entries,
        'Lattice parameter a': entries,
        'Lattice parameter b': entries,
        'Lattice parameter c': entries,
        'Band gap [eV]': entries,
        'Final energy [eV]': entries,
        'CPU time [s]': entries
    }
)

In [521]:
from pymatgen.io import vasp

for i, job in enumerate(jobs):
    print(f'Parsing {job}')
    run = vasp.Vasprun(f'{temp_localfolder}{job}/vasprun.xml')
    outcar = vasp.Outcar(f'{temp_localfolder}{job}/OUTCAR')
    
    results.loc[i, 'Job name'] = job
    results.loc[i, 'k-points'] = str(run.kpoints.kpts[0]).replace(', ', 'x').strip('[](),')
    results.loc[i, '# k-points'] = len(run.actual_kpoints),
    results.loc[i, 'Energy cutoff [eV]'] = run.incar['ENCUT'],
    results.loc[i, 'Final energy [eV]'] = run.final_energy/len(run.atomic_symbols),
    results.loc[i, 'Band gap [eV]'] = run.eigenvalue_band_properties[0],
    results.loc[i, 'Lattice parameter a'] = run.final_structure.lattice.abc[0],
    results.loc[i, 'Lattice parameter b'] = run.final_structure.lattice.abc[1],
    results.loc[i, 'Lattice parameter c'] = run.final_structure.lattice.abc[2],
    results.loc[i, 'CPU time [min]'] = outcar.run_stats['Total CPU time used (sec)']/60

Parsing bulk_LPS_mesh_4x4x4_500eV


/Users/dominicwelti/opt/anaconda3/lib/python3.9/site-packages/pymatgen/io/vasp/inputs.py:1837: UserWarning:

POTCAR with symbol Li and functional
PBE has a SHA256 hash defined,
but the computed hash differs.
YOUR POTCAR FILE HAS BEEN CORRUPTED AND SHOULD NOT BE USED!

/Users/dominicwelti/opt/anaconda3/lib/python3.9/site-packages/pymatgen/io/vasp/inputs.py:1837: UserWarning:

POTCAR with symbol P and functional
PBE has a SHA256 hash defined,
but the computed hash differs.
YOUR POTCAR FILE HAS BEEN CORRUPTED AND SHOULD NOT BE USED!

/Users/dominicwelti/opt/anaconda3/lib/python3.9/site-packages/pymatgen/io/vasp/inputs.py:1837: UserWarning:

POTCAR with symbol S and functional
PBE has a SHA256 hash defined,
but the computed hash differs.
YOUR POTCAR FILE HAS BEEN CORRUPTED AND SHOULD NOT BE USED!



Parsing bulk_LPS_mesh_8x10x14_500eV


/Users/dominicwelti/opt/anaconda3/lib/python3.9/site-packages/pymatgen/io/vasp/inputs.py:1837: UserWarning:

POTCAR with symbol Li and functional
PBE has a SHA256 hash defined,
but the computed hash differs.
YOUR POTCAR FILE HAS BEEN CORRUPTED AND SHOULD NOT BE USED!

/Users/dominicwelti/opt/anaconda3/lib/python3.9/site-packages/pymatgen/io/vasp/inputs.py:1837: UserWarning:

POTCAR with symbol P and functional
PBE has a SHA256 hash defined,
but the computed hash differs.
YOUR POTCAR FILE HAS BEEN CORRUPTED AND SHOULD NOT BE USED!

/Users/dominicwelti/opt/anaconda3/lib/python3.9/site-packages/pymatgen/io/vasp/inputs.py:1837: UserWarning:

POTCAR with symbol S and functional
PBE has a SHA256 hash defined,
but the computed hash differs.
YOUR POTCAR FILE HAS BEEN CORRUPTED AND SHOULD NOT BE USED!



Parsing bulk_LPS_LREAL_auto_mesh_2x2x2_500eV
Parsing bulk_LPS_mesh_LREAL_2x2x2_500eV
Parsing bulk_LPS_mesh_LREAL_6x10x12_500eV
Parsing bulk_LPS_mesh_7x9x15_500eV
Parsing bulk_LPS_mesh_LREAL_7x9x15_500eV
Parsing bulk_LPS_LREAL_auto_mesh_2x2x4_500eV
Parsing bulk_LPS_mesh_LREAL_2x2x4_500eV
Parsing bulk_LPS_mesh_8x8x8_500eV
Parsing bulk_LPS_mesh_3x5x6_500eV
Parsing bulk_LPS_mesh_LREAL_6x6x6_500eV
Parsing bulk_LPS_mesh_LREAL_4x5x8_500eV
Parsing bulk_LPS_mesh_1x2x2_500eV
Parsing bulk_LPS_mesh_1x1x1_500eV
Parsing bulk_LPS_mesh_3x4x6_500eV
Parsing bulk_LPS_mesh_2x2x4_500eV
Parsing bulk_LPS_mesh_LREAL_5x7x11_500eV
Parsing bulk_LPS_mesh_6x8x13_500eV
Parsing bulk_LPS_mesh_5x7x11_500eV
Parsing bulk_LPS_mesh_2x2x2_500eV
Parsing bulk_LPS_mesh_LREAL_6x8x13_500eV
Parsing bulk_LPS_mesh_LREAL_4x4x4_500eV
Parsing bulk_LPS_mesh_LREAL_1x2x2_500eV
Parsing bulk_LPS_LREAL_auto_mesh_1x2x2_500eV
Parsing bulk_LPS_mesh_4x5x8_500eV
Parsing bulk_LPS_mesh_6x10x12_500eV
Parsing bulk_LPS_mesh_LREAL_8x10x14_500eV
Parsi

In [478]:
# sort values
results = results.sort_values('Energy cutoff [eV]').reset_index(drop=True)

In [406]:
results['Change to previous value [eV]'] = 0

for i, job in enumerate(results['Job name']):
    if i==0:
        results.loc[i, 'Change to previous value [eV]'] = None
    else:
        results.loc[i, 'Change to previous value [eV]'] = abs(results.loc[i, 'Final energy [eV]'] - results.loc[i-1, 'Final energy [eV]'])

In [479]:
max_value = float(results.tail(1)['Final energy [eV]'])

In [451]:
# graphing for energy cutoff
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=3, cols=1, shared_xaxes=True)

fig.append_trace(go.Scatter(
     x=results['Energy cutoff [eV]'],
     y=results['Final energy [eV]'],
     showlegend=False
    ),
    row=1, col=1
)
fig.append_trace(go.Scatter(
     x=results['Energy cutoff [eV]'],
     y=results['Change to previous value [eV]'],
     showlegend=False
    ),
    row=2, col=1
)
fig.append_trace(go.Scatter(
     x=results['Energy cutoff [eV]'],
     y=results['CPU time [min]'],
     showlegend=False
    ),
    row=3, col=1
)

fig.update_layout(
    title_text = 'Convergence test - Energy cutoff',

)
fig.update_xaxes(title_text='Energy cutoff [eV]', row=3, col=1)
fig.update_yaxes(title_text='Final energy per atom [eV]', row=1, col=1, range=[max_value-0.01,max_value+0.01])
fig.update_yaxes(title_text='Change to previous point [eV]', row=2, col=1,range=[0,0.015])
fig.update_yaxes(title_text='CPU time [min]', row=3, col=1)
fig.add_hrect(y0=max_value-0.001,y1=max_value+0.001, row=1, col=1, line_width=0, fillcolor='yellow', opacity=0.2, annotation_text="1 meV range", annotation_position="top left")
fig.add_hrect(y0=-0.001,y1=0.001, row=2, col=1, line_width=0, fillcolor='yellow', opacity=0.2, annotation_text="1 meV range", annotation_position="top left")

In [336]:
# save plot as HTML file
fig.write_html('/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/Covergence test bulk/E_cutoff.html')

In [526]:
# sort values
results = results.sort_values('# k-points').reset_index(drop=True)

In [532]:
results['Real space'] = 0

for i, name in enumerate(results['Job name']):
    if 'LREAL' in name:
        if 'auto' in name:
            results.loc[i, 'Real space'] = 'auto'
        else:
            results.loc[i, 'Real space'] = 'on'
    else:
        results.loc[i, 'Real space'] = 'off'

results_off = results.query('`Real space` == "off"', engine='python')
results_on = results.query('`Real space` == "on"', engine='python')
results_auto = results.query('`Real space` == "auto"', engine='python')

In [533]:
max_value = float(results_off.tail(1)['Final energy [eV]'])

In [534]:
# graphing for k-points
fig = make_subplots(rows=2, cols=2, shared_xaxes=True)

fig.append_trace(go.Scatter(
            x=results_off['# k-points'],
            y=results_off['Final energy [eV]'],
            name='Reciprocal space energy',
            hovertemplate=
                "%{text}"+"<br>Number of k-points: %{x}"+"<br>Final energy: %{y}",
            text = results_off['k-points'],
            showlegend=False
        ),
        row=1, col=1
        )
fig.append_trace(go.Scatter(
            x=results_off['# k-points'],
            y=results_off['CPU time [min]'],
            name='Reciprocal space computation time',
            hovertemplate=
                "%{text}"+"<br>Number of k-points: %{x}"+"<br>CPU time: %{y}",
            text = results_off['k-points'],
            showlegend=False
        ),
        row=2, col=1
        )
fig.update_layout(
        title_text = 'Convergence test - k-point mesh',
        )

fig.update_xaxes(title_text='Number of k-points', row=2, col=1)
fig.update_yaxes(title_text='Final energy per atom [eV]', row=1, col=1, range=[max_value-0.01,max_value+0.01])
fig.update_yaxes(title_text='CPU time [min]', row=2, col=1)
fig.add_hrect(y0=max_value-0.001,y1=max_value+0.001, row=1, col=1, line_width=0, fillcolor='yellow', opacity=0.2, annotation_text="1 meV range", annotation_position="top left")

fig.append_trace(go.Scatter(
            x=results_on['# k-points'],
            y=results_on['Final energy [eV]'],
            name='Real space energy',
            hovertemplate=
                "%{text}"+"<br>Number of k-points: %{x}"+"<br>Final energy: %{y}",
            text = results_on['k-points'],
            showlegend=False
        ),
        row=1, col=2
        )
fig.append_trace(go.Scatter(
            x=results_on['# k-points'],
            y=results_on['CPU time [min]'],
            name='Real space computation time',
            hovertemplate=
                "%{text}"+"<br>Number of k-points: %{x}"+"<br>CPU time: %{y}",
            text = results_on['k-points'],
            showlegend=False
        ),
        row=2, col=2
        )

fig.update_xaxes(title_text='Number of k-points', row=2, col=2)
fig.update_yaxes(title_text='Final energy per atom [eV]', row=1, col=2, range=[max_value-0.01,max_value+0.01])
fig.update_yaxes(title_text='CPU time [min]', row=2, col=2)
fig.add_hrect(y0=max_value-0.001,y1=max_value+0.001, row=1, col=2, line_width=0, fillcolor='yellow', opacity=0.2, annotation_text="1 meV range", annotation_position="top left")

In [539]:
results['Real space'].unique()

array(['auto', 'off', 'on'], dtype=object)

In [544]:
# graphing 1x2
fig = make_subplots(rows=2, cols=1, shared_xaxes=True)

for setting in results['Real space'].unique():
    subset = results.query('`Real space`==@setting')
    fig.append_trace(go.Scatter(
                x=subset['# k-points'],
                y=subset['Final energy [eV]'],
                name=f'LREAL: {setting}',
                hovertemplate=
                    "%{text}"+"<br>Number of k-points: %{x}"+"<br>Final energy: %{y}",
                text = subset['k-points']
            ),
            row=1, col=1
            )
    fig.append_trace(go.Scatter(
                x=subset['# k-points'],
                y=subset['CPU time [min]'],
                name=f'LREAL: {setting}',
                hovertemplate=
                    "%{text}"+"<br>Number of k-points: %{x}"+"<br>CPU time: %{y}",
                text = subset['k-points']
            ),
            row=2, col=1
            )
fig.update_layout(
        title_text = 'Convergence test - k-point mesh',
        )

fig.update_xaxes(title_text='Number of k-points', row=2, col=1)
fig.update_yaxes(title_text='Final energy per atom [eV]', row=1, col=1, range=[max_value-0.01,max_value+0.01])
fig.update_yaxes(title_text='CPU time [min]', row=2, col=1)
fig.add_hrect(y0=max_value-0.001,y1=max_value+0.001, row=1, col=1, line_width=0, fillcolor='yellow', opacity=0.2, annotation_text="1 meV range", annotation_position="top left")

In [545]:
# save plot as HTML file
fig.write_html('/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/Covergence test bulk/k_mesh_energy_settings_LREAL.html')

In [536]:
# graphing for k-points - band gap
fig = make_subplots(rows=2, cols=2, shared_xaxes=True)

fig.append_trace(go.Scatter(
            x=results_off['# k-points'],
            y=results_off['Band gap [eV]'],
            name='Reciprocal space band gap',
            hovertemplate=
                "%{text}"+"<br>Number of k-points: %{x}"+"<br>Band gap: %{y}",
            text = results_off['k-points'],
            showlegend=False
        ),
        row=1, col=1
        )
fig.append_trace(go.Scatter(
            x=results_off['# k-points'],
            y=results_off['CPU time [min]'],
            name='Reciprocal space computation time',
            hovertemplate=
                "%{text}"+"<br>Number of k-points: %{x}"+"<br>CPU time: %{y}",
            text = results_off['k-points'],
            showlegend=False
        ),
        row=2, col=1
        )
fig.update_layout(
        title_text = 'Band gap - k-point mesh',
        )

fig.update_xaxes(title_text='Number of k-points', row=2, col=1)
fig.update_yaxes(title_text='Band gap [eV]', row=1, col=1)
fig.update_yaxes(title_text='CPU time [min]', row=2, col=1)

fig.append_trace(go.Scatter(
            x=results_on['# k-points'],
            y=results_on['Band gap [eV]'],
            name='Real space band gap',
            hovertemplate=
                "%{text}"+"<br>Number of k-points: %{x}"+"<br>Final energy: %{y}",
            text = results_on['k-points'],
            showlegend=False
        ),
        row=1, col=2
        )
fig.append_trace(go.Scatter(
            x=results_on['# k-points'],
            y=results_on['CPU time [min]'],
            name='Real space computation time',
            hovertemplate=
                "%{text}"+"<br>Number of k-points: %{x}"+"<br>CPU time: %{y}",
            text = results_on['k-points'],
            showlegend=False
        ),
        row=2, col=2
        )

fig.update_xaxes(title_text='Number of k-points', row=2, col=2)
fig.update_yaxes(title_text='Band gap [eV]', row=1, col=2)
fig.update_yaxes(title_text='CPU time [min]', row=2, col=2)

In [537]:
# save plot as HTML file
fig.write_html('/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/Covergence test bulk/k_mesh_bandgap.html')